## Exploratory Data Analysis
This notebook will go into some brief EDA and a baseline Xgboost model for the transactions and identity data provided by the Kaggle competition, IEEE-CIS Fraudulent Transactions.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import datetime
import missingno as msno
import lightgbm as lgb
import xgboost as xgb
from sklearn import preprocessing
import gc
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from time import time
plt.rcParams['figure.figsize'] = [10, 5]

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

### The Data
In this competition we are predicting the probability that an online transaction is fraudulent, as denoted by the binary target isFraud.

The data is broken into two files identity and transaction, which are joined by TransactionID.

Note: Not all transactions have corresponding identity information.

Categorical Features - Transaction

- ProductCD: product code, the product for each transaction
- card1 - card6: payment card information, such as card type, card category, issue bank, country, etc.
- addr1, addr2: billing region, billing country
- P_emaildomain: purchaser email domain
- R_emaildomain: recipient email domain
- M1 - M9: match, such as names on card and address, etc.

Categorical Features - Identity
Variables in this table are identity information – network connection information (IP, ISP, Proxy, etc) and digital signature (UA/browser/os/version, etc) associated with transactions. 
They're collected by Vesta’s fraud protection system and digital security partners.
- DeviceType: 
- DeviceInfo
- id_12 - id_38

Numerical features
- TransactionAMT: transaction payment amount in USD. Non-US transactions have an exchange rate applied, so are not exact, have a number of extra dp's. This is potentially already marked in ProductCD as C.
- TrasactionDT: timedelta from a given reference datetime (not an actual timestamp). TransactionDT first value is 86400, which corresponds to the number of seconds in a day (60 * 60 * 24 = 86400) so I think the unit is seconds. Using this, we know the data spans 6 months, as the maximum value is 15811131, which would correspond to day 183. *Might be good to split train/validation sets by time, since train/test is split by time
- dist: distances between (not limited) billing address, mailing address, zip code, IP address, phone area, etc.
- C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc.
- Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.
- D1-D15: timedelta, such as days between previous transaction, etc.

The TransactionDT feature is a timedelta from a given reference datetime (not an actual timestamp, but we can potentially still use this to build time dependent features).

#### Definition of Fraud
Below is the definition of fraud by one of the Vesta team organisers:
"The logic of our labeling is define reported chargeback on the card as fraud transaction (isFraud=1) and transactions posterior to it with either user account, email address or billing address directly linked to these attributes as fraud too. If none of above is reported and found beyond 120 days, then we define as legit transaction (isFraud=0).
However, in real world fraudulent activity might not be reported, e.g. cardholder was unaware, or forgot to report in time and beyond the claim period, etc. In such cases, supposed fraud might be labeled as legit, but we never could know of them. Thus, we think they're unusual cases and negligible portion."

In [ ]:
# identity_train = pd.read_csv("/kaggle/input/ieee-fraud-detection/train_identity.csv")
# identity_test = pd.read_csv("/kaggle/input/ieee-fraud-detection/test_identity.csv")
# transaction_train = pd.read_csv("/kaggle/input/ieee-fraud-detection/train_transaction.csv")
# transaction_test = pd.read_csv("/kaggle/input/ieee-fraud-detection/test_transaction.csv")

In [ ]:
# print(identity_train.shape)
# print(identity_test.shape)
# print(transaction_train.shape)
# print(transaction_test.shape)

In [ ]:
# identity_train.head()

In [ ]:
# transaction_train.head()

The transactionID is a unique key throughout the datasets. There are 590k transactions, all unique. In terms of identity, we only have identity data for 144233 out of those 590k transactions (~24%)

The 'isFraud' flag is the target variable. As expected, a heavy imbalance, with about 96.5% of non-fraud transactions.

In [ ]:
# transaction_train['isFraud'].value_counts(normalize=True).to_frame()

In [ ]:
# print(transaction_train['TransactionID'].nunique())
# print(identity_train['TransactionID'].nunique())

In [ ]:
# del transaction_test,identity_test, transaction_train, identity_train
# gc.collect()

Get the train and test sets from my earlier pickle files

In [ ]:
train_full = pd.read_pickle('/kaggle/input/ieee-cis-fraudulent-transactions-data-prep/train_full.pkl')
test_full = pd.read_pickle('/kaggle/input/ieee-cis-fraudulent-transactions-data-prep/test_full.pkl')

In [ ]:
train_full.info(verbose=True, null_counts=True)

Summary of uniques for all variables in dataset

In [ ]:
for col, values in train_full.iteritems():
    num_uniques = values.nunique()
    print ('{name}: {num_unique}'.format(name=col, num_unique=num_uniques))
    print (values.unique())
    print ('\n')

#### Missingness Plot

In [ ]:
pd.set_option('display.max_columns', 999)
def missing_data(df) :
    count = df.isnull().sum()
    percent = (df.isnull().sum()) / (df.isnull().count()) * 100
    total = pd.concat([count, percent], axis=1, keys = ['Count', 'Percent'])
    types = []
    for col in df.columns :
        dtypes = str(df[col].dtype)
        types.append(dtypes)
    total['dtypes'] = types
    
    return np.transpose(total)

missing_df = missing_data(train_full)
missing_df

In [ ]:
# draw missingness heatmap
msno.matrix(train_full)

Pretty sparse dataset, with alot of NAs. Let's do some more EDA.

### Numerical Variables
Let's start with a 5 number summary for numerical variables:

In [ ]:
train_full[train_full.columns[train_full.columns.str.contains('V')]].describe()

The features engineering by Vesta are very difficult to decipher and all have very different ranges.

Next we'll look at the distributions of dist columns vs Fraud:

In [ ]:
train_full[train_full.columns[train_full.columns.str.contains('dist')]].describe()

In [ ]:
def plot_dist(variable):
    sns.distplot(train_full[variable][train_full.isFraud==1], kde=True, hist=False,label="fraud")
    sns.distplot(train_full[variable][train_full.isFraud==0], kde=True, hist=False,label="notfraud")
    plt.legend(prop={'size': 10}, title = 'IsFraud')
    plt.title(variable +' vs IsFraud')

plot_dist('dist1')

In [ ]:
plot_dist('dist2')

Mostly minor differences in distribution, although it looks like a high portion of fraud where the dist1 is smaller

In [ ]:
plot_dist('TransactionAmt')

Definitely some relationship between transaction amount and fraud.

In [ ]:
plot_dist('TransactionDT')

The peak in notfraud distribution could be caused more by the total transactions increasing during christmas. Anyhow, the date itself won't be a feature due to overfit and leakage problems, but we could probably use a day of week and hour of day feature

Variables C1-C14

In [ ]:
train_full[train_full.columns[train_full.columns.str.contains('C')]].describe()

Also various numbers given in the C variables. Hard to decipher since it is masked

In [ ]:
cols = [col for col in train_full.columns if 'C' in col or col=="isFraud"]

(train_full[cols].corr())


Not alot of correlation between C variables and Fraud, but the C variables have alot of correlation with each other. Eg. (C2,C2,C4,C6,C7,C8,C10,C11,C12,C14), (C5,C9)

In [ ]:
cols = [col for col in train_full.columns if 'D' in col or col=="isFraud"]

(train_full[cols].corr())


Some minor correlations between IsFraud and D variables. Again a number of correlations between the D variables themselves. Most promising are D7 and D8. And below, we see that the total not fraud transactions spike at a low number and also at a random number ~360?

In [ ]:
plot_dist('D7')

In [ ]:
plot_dist('D8')

### Categorical Features

In [ ]:
sns.countplot(x='ProductCD',hue="isFraud",data=train_full)

In [ ]:
pd.crosstab(train_full.ProductCD,train_full.isFraud, normalize='index')

Largest portion of fraud is happening with ProductCD = C

In [ ]:
for col, values in train_full[train_full.columns[train_full.columns.str.contains('card')]].iteritems():
    num_uniques = values.nunique()
    print ('{name}: {num_unique}'.format(name=col, num_unique=num_uniques))
    print (values.unique())
    print ('\n')

Alot of categories in the card variables. Card 4 and 6 have a much smaller number and recognisable categories. And seen below there are many small card1 catgories which have never had fraud. This could result in some overfitting or noise. We will need a way to treat the rare categories. Maybe group them together?

In [ ]:
pd.crosstab(train_full.card1,train_full.isFraud)

In [ ]:
pd.crosstab(train_full.card4,train_full.isFraud,normalize='index')

More fraud happens in discover, but thats not a very large category. FOr mastercard and visa, theres very little difference.

In [ ]:
train_full.groupby('card1').size().sort_values(ascending=False).plot.bar()

In [ ]:
train_full.groupby('card2').size().sort_values(ascending=False).plot.bar()

In [ ]:
train_full.groupby('card3').size().sort_values(ascending=False).plot.bar()

In [ ]:
train_full.groupby('card5').size().sort_values(ascending=False).plot.bar()

Alot of rare categories. It would be good to take only top 20 categories and group the rest in "Other"

In [ ]:
pd.crosstab(train_full.card6,train_full.isFraud)

In [ ]:
test_full.groupby('card6').size()

Not many cards in the charge card or debit or credit category. These don't have Fraud example either. This could be bad if there are any of these in the test set. In the test set we only have 1 charge card and none in 'debit or credit'

In [ ]:
train_full.groupby('addr1').size().sort_values(ascending=False).plot.bar()

In [ ]:
train_full.groupby('addr2').size().sort_values(ascending=False).plot.bar()

In [ ]:
train_full.groupby(['addr2', 'addr1']).size()

A large number of billing regions, less for billing country. Still, alot of rare categories in both. The biggest country is likely the US. There seems to be only 437 unique combos of region+country?

In [ ]:
train_full.groupby('P_emaildomain').size().sort_values(ascending=False).plot.bar()

In [ ]:
pd.crosstab(train_full.P_emaildomain,train_full.isFraud,normalize='index')

In [ ]:
train_full.groupby('R_emaildomain').size().sort_values(ascending=False).plot.bar()

In [ ]:
pd.crosstab(train_full.R_emaildomain,train_full.isFraud,normalize='index')

The email domain as expected are mostly gmail, hotmail and yahoo. The most fraud happens on Gmail.

In [ ]:
for col, values in train_full[train_full.columns[train_full.columns.str.contains('M')]].iteritems():
    num_uniques = values.nunique()
    print ('{name}: {num_unique}'.format(name=col, num_unique=num_uniques))
    print (values.unique())
    print ('\n')

Only a True/False matching check. Perhaps if there is a number of failed matching checks this could flag as fraud?

In [ ]:
pd.crosstab(train_full.M1,train_full.isFraud)

In [ ]:
pd.crosstab(train_full.M2,train_full.isFraud,normalize='index')

In [ ]:
pd.crosstab(train_full.M3,train_full.isFraud,normalize='index')

In [ ]:
pd.crosstab(train_full.M4,train_full.isFraud,normalize='index')

In [ ]:
pd.crosstab(train_full.M5,train_full.isFraud,normalize='index')

In [ ]:
pd.crosstab(train_full.M6,train_full.isFraud,normalize='index')

In [ ]:
pd.crosstab(train_full.M7,train_full.isFraud,normalize='index')

In [ ]:
pd.crosstab(train_full.M8,train_full.isFraud,normalize='index')

In [ ]:
pd.crosstab(train_full.M9,train_full.isFraud,normalize='index')

Not a heap of correlation between each the M variables with the Fraud. Might be useful to look at multiple M variables with Fraud. Also M4 seems to be a combination of M0,M1 and M2. M4==M2 seems to have a much higher rate of Fraud.

In [ ]:
pd.crosstab(train_full.DeviceType,train_full.isFraud,normalize='index')

More fraud happens on mobile

In [ ]:
train_full.groupby('DeviceInfo').size()[:20].sort_values(ascending=False).plot.bar()

Top 20 Device Infos

In [ ]:
for col, values in train_full[train_full.columns[train_full.columns.str.contains('id')]].iteritems():
    num_uniques = values.nunique()
    print ('{name}: {num_unique}'.format(name=col, num_unique=num_uniques))
    print (values.unique())
    print ('\n')

id_30: Operating system; id_31: browser; id_23: Ip proxy info; id_33: Screen resolution; id_35-38: True/false; id_27-29: Found/notfound

In [ ]:
pd.crosstab(train_full.id_23,train_full.isFraud,normalize='index')

In [ ]:
pd.crosstab(train_full.id_35,train_full.isFraud,normalize='index')

In [ ]:
pd.crosstab(train_full.id_36,train_full.isFraud,normalize='index')

In [ ]:
pd.crosstab(train_full.id_37,train_full.isFraud,normalize='index')

In [ ]:
pd.crosstab(train_full.id_38,train_full.isFraud,normalize='index')

In [ ]:
pd.crosstab(train_full.id_27,train_full.isFraud,normalize='index')

In [ ]:
pd.crosstab(train_full.id_28,train_full.isFraud,normalize='index')

In [ ]:
pd.crosstab(train_full.id_29,train_full.isFraud,normalize='index')

Anonymous proxy is much more likely to have fraud than the other proxy ips, but theres not that many proxy ips to start with. Probably the fact that they have a proxy ip is a good feature?
id_35-38 seem to have good correlation too. False has more chance of fraud. id_27-29: clearly found is more likely to be Fraud

### Feature Engineering
- build time of day/ week, month features (D9 is already a time of day feature, but because it has alot of NAs, better to create using the timedelta variable)
- Hour and TransactionPerHour
- Number of failed matching checks
- proxy ip or not
- create category for NAs

In [ ]:
# # Time dependent features
# # https://www.kaggle.com/fchmiel/day-and-time-powerful-predictive-feature
# train_full['Transaction_day_of_week'] = np.floor((train_full['TransactionDT'] / (3600 * 24) - 1) % 7)
# test_full['Transaction_day_of_week'] = np.floor((test_full['TransactionDT'] / (3600 * 24) - 1) % 7)
# train_full['Transaction_hour'] = np.floor(train_full['TransactionDT'] / 3600) % 24
# test_full['Transaction_hour'] = np.floor(test_full['TransactionDT'] / 3600) % 24